## Imports

In [1]:
import pandas
import numpy as np
import cv2
from scipy.stats import norm
import os
from os import listdir
from os.path import isfile, join
import matplotlib.mlab as mlab
import math
import xlsxwriter
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from scipy.stats import norm
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from scipy import ndimage

#importing pandas as pd
import pandas as pd
from numpy import genfromtxt
from io import StringIO
import csv

## Lee la carpeta con las fotos y genera dos excel con el registro de las mismas regresa un arreglo con la informacioón de todas las fotos

In [2]:
def leer(nombre):
    onlyfiles = [ f for f in listdir(nombre) if isfile(join(nombre,f)) ]
    
    a = onlyfiles[0].split('_')#separa el nombre para que tener los
    #datos d ela imagen
    b = a
    b = np.array(b)
    b = np.reshape(b, (1, 4))

    for n in range(1, len(onlyfiles)):
        aux = onlyfiles[n].split('_')
        aux = np.array(aux)
        aux = np.reshape(aux, (1, 4))
        b = np.concatenate((b,aux), axis = 0)
    #crear los excels, uno para cada tipo de foto
    #afuero o cabina
    outWorkbook = xlsxwriter.Workbook("afuera.xlsx")
    outSheet = outWorkbook.add_worksheet()
    outWorkbook2 = xlsxwriter.Workbook("cabina.xlsx")
    outSheet2 = outWorkbook2.add_worksheet()
    
        
    fila = 1
    #Tomas los valores únicos de corte, compra y muestra
    for compra in np.unique( b[:,1] ):
    
        #print(compra)
        idx = np.where( b[:,1] == compra )[0]
        bcompra = b[idx,:]
        for muestra in np.unique( bcompra[:,2] ):
        
            #print(muestra)
            idx = np.where( bcompra[:,2] == muestra)[0]
            bmuestra = bcompra[idx,:]
        
            aux =  fila
            aux = str(aux)
        
            outSheet.write('A'+aux, compra[0])
            outSheet.write('B'+aux, compra[1:])
            outSheet.write('C'+aux, muestra)
                
            outSheet2.write('A'+aux, compra[0])
            outSheet2.write('B'+aux, compra[1:])
            outSheet2.write('C'+aux, muestra)
            
            orden = set()
            for fecha in bmuestra:
                aux = int(fecha[0])
                orden.add(aux)
                
            orden = np.array(list(orden))
            orden.sort()
            
            #print(orden)
            
            cont = 0
            posi = {}
            for fecha in orden:
                posi[fecha] = cont;
                cont = cont+1
                
            #print(posi)
                
            for fecha in bmuestra:
                #acomodamos en el excel según su fecha
                if(fecha[3] == 'a.JPG'):
                        
                        
                    #print(fecha[0][6:8])
                    #pos = 3 + min- ((int(fecha[0]))%1000)
                    letra = ord('A')
                    letra = letra + 3 + posi[int(fecha[0])]
                    letra = chr(letra)
                    aux = fila
                    aux = str(fila)
                    outSheet.write(letra+aux, fecha[0])
                    
                else:
                        
                    letra = ord('A')
                    letra = letra + 3 + posi[int(fecha[0])]
                    letra = chr(letra)
                    aux = fila
                    aux = str(fila)                
                    outSheet2.write(letra+aux, fecha[0])
                
            fila = fila+1
    
    outWorkbook.close()
    outWorkbook2.close()#guardar los excels
    return b

## Segmentación de imágenes

In [3]:
def get_largest_component( msk ) : 
    n_components , labels , stats , _ = cv2.connectedComponentsWithStats( msk.astype( np.uint8 ) , connectivity = 4) 
    max_area = 0
    for i in range( 1 , n_components ) : 
        area = stats[ i , cv2.CC_STAT_AREA ] 
        if area > max_area:
            max_area = area
            msk = labels == i
    return msk
#crear una máscara 
def get_reds2(img) : 
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    msk = np.bitwise_or(img[ :  , :  , 2] < 0 , img[ :  , :  , 2] > 0)
    # usamos el valor y la imagen original
    img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    return msk
#convierte en negro lo que no nos interesa en la máscara
def blackout( img , msk ) : 
    hsv = cv2.cvtColor( img , cv2.COLOR_RGB2HSV ) 
    h , s , v = cv2.split( hsv ) 
    v[ msk ] = 0
    hsv = cv2.merge(( h , s , v ) ) 
    img = cv2.cvtColor( hsv , cv2.COLOR_HSV2RGB ) 
    return img
#recibe la carpeta y el nombre del archivo que deseamos segmentar
#guarda la imagen y la regresa
def mascaraa(path1,path2,name):
    
    img = cv2.imread(join(path1,name))
    img2 = np.copy(img)
    
    nrows,ncols,_=img.shape
    Ximg = np.reshape(img,(nrows*ncols,3))
    
    y = model.predict(Ximg)
    yimg = np.reshape(y,(nrows,ncols))
    
    imgcarne = img.copy()
    imgcarne[yimg==0,:] = 0
    img = imgcarne
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    '''
    plt.figure()
    plt.imshow(img)
    plt.show()
    '''
    mask =  get_reds2(img)
    global new
    new = get_largest_component(mask)
    new = ndimage.binary_fill_holes(new).astype(int)
    new[new != 0] = 255
    res = new < 255
    x,y = np.where(res)
    '''
    plt.figure()
    plt.imshow(new,cmap="gray")
    plt.show()
    '''
    #x,y = np.where(x)
    #print(x.shape,y.shape)
    
    img2[x,y] = 0
    '''
    plt.figure()
    plt.imshow(img2)
    plt.show()
    '''
    
    os.chdir(path2)
    cv2.imwrite(name[0:-3] + "PNG", img2)
    return img2

# guarda la segmentación
#en una carpeta llamada segmentacion
#recibe a fila del excel que se desea segmentar y el tipo de
#corte que es
def pormuestra(arr,t,path1,path2):#t es si es cabina o afuera
    res = []
    for i in range(len(arr) - 3):
        aux = arr[1];
        aux = '0'+aux;
        ayuda = int(float(arr[3+i]))
        ayuda = str(ayuda)
        aux = ayuda+'_'+arr[0]+aux+'_'+arr[2]+'_'+t+'.JPG';
        res.append(aux)
    
    #print(res)
    #imgs = []
    for nombre in res:
        print("Procesando:",nombre)
        img = mascaraa(path1,path2,nombre)
        

## Entrenamiento SVC

In [4]:
carne = cv2.imread('fotos_para_entrenar/carne.JPG')
fondo = cv2.imread('fotos_para_entrenar/fondo.JPG') #fotos de la carne y el fondo

nrows,ncols,_= carne.shape
Xcarne = np.reshape(carne,(nrows*ncols,3))
nrows,ncols,_= fondo.shape
Xfondo = np.reshape(fondo,(nrows*ncols,3))#reshape

idx = np.random.permutation(nrows*ncols)[:5000]
Xfondo = Xfondo[idx,:]
Xcarne = Xcarne[idx,:]

X = np.concatenate((Xcarne,Xfondo),axis=0)
y = np.zeros(10000) #para distiguir cual es fondo y cual carne
y[:5000] = 1

model = SVC()
model.fit(X,y)#entrenamiento

SVC()

## Algoritmo

In [5]:
def AlgoritmoSegmentacion(Path, Carpeta_imagenes, Carpeta_segmentadas):
    os.chdir(Path) #Accede a la dirección agregada
    leer(Carpeta_imagenes) #Se leen las imágenes de la carpeta y se generan 2 excels
    
    try: #Se crea la carpeta en donde se van a guardar las imágenes si no existe
        os.mkdir(Carpeta_segmentadas) 
    except:
        print("Ya existe la carpeta "+ Carpeta_segmentadas)
        
    read_file = pd.read_excel ("cabina.xlsx") #Lee el excel cabina que genera la función leer
    read_file.to_csv ("cabina.csv", index = None,header=True)

    csvfile = open('cabina.csv', 'r')
    reader = csv.reader(csvfile, delimiter=',')
    my_list = list(reader)
    my_list = np.array(my_list)
    my_list[0][1] = my_list[0][1][1:]
    
    for i in range(my_list.shape[0]):
        pormuestra(my_list[i],'c', Path + Carpeta_imagenes, Path + Carpeta_segmentadas)#'c' porque es en cabina, segmenta las imágenes y las guarda
    

In [ ]:
Path = r'C:\Users\PC Gamer\Desktop\Up7mo\Carnes' + chr(92) #El path de donde se va a ejecutar el proyecto, chr(92) es para agregar \
Carpeta_imagenes = "imagenes_calibracion" #Nombre de la carpeta de donde se van a leer las imágenes
Carpeta_segmentadas = "segmentadas" #Nombre de la carpeta donde se van a guardar las imágenes, no necesariamente tiene que existir
AlgoritmoSegmentacion(Path, Carpeta_imagenes, Carpeta_segmentadas)

Ya existe la carpeta segmentadas
Procesando: 20210830_A01_1_c.JPG
